<div style='text-align: center;'>
<img src="images/math60082-banner.png" alt="image" width="80%" height="auto">
</div>

# Lab Workbook - Week 10


In [ ]:
# To get started import some libraries
import numpy
from scipy.linalg import solve_banded
import matplotlib.pyplot as plt

In [ ]:
def finiteDiff_AmericanPut( X,T, r,sigma, SU,imax,jmax):
    ## calculate the step size and then setup storage for the value
    dS = SU / jmax
    dt = T / imax

    S = numpy.zeros(jmax+1)
    t = numpy.zeros(imax+1)
    vNew = numpy.zeros(jmax+1)
    vOld = numpy.zeros(jmax+1)

    # calculate the values of S_j and t^i and check they work as expected
    for j in range(jmax+1):
        S[j] = j*dS

    for i in range(imax+1):
        t[i] = i*dt
        
    # first enter the value of the option at expiry
    for j in range(jmax+1):
        vOld[j] = max( X - S[j] , 0.)
        vNew[j] = max( X - S[j] , 0.)
        
    # next loop back through time
    for i in range(imax-1,-1,-1):
        # apply boundary condition at S=0
        vNew[0] = X
        for j in range(1,jmax):
            # input explicit finite difference formula
            A = 0.5*sigma*sigma*j*j*dt + 0.5*r*j*dt
            B = 1.0 - sigma*sigma*j*j*dt
            C = 0.5*sigma*sigma*j*j*dt - 0.5*r*j*dt
            # use vNew[j] for V_j^i and vOld[j] for V_j^{i+1}
            vNew[j] = max( (A*vOld[j+1] + B*vOld[j] + C*vOld[j-1]) / ( 1 + r*dt) , X - S[j] )
        # apply boundary condition at S=S_U
        vNew[jmax] = 0.0
        
        # set old values equal to new ones
        # MAKE SURE THIS IS A DEEP COPY!
        vOld = numpy.copy(vNew)
    
    return S,vNew

In [ ]:
S,vNew = finiteDiff_AmericanPut(10,1,0.05 , 0.4,50,1000,50)

# check a plot to see what it looks like
plt.xlim(0,20)
plt.plot(S,vNew)

In [ ]:
# penalty method solver
def crankNicolson_AmerPut( X,T ,r,sigma , SU,imax,jmax , rho,tol,maxiter ):
    ## calculate the step size and then setup storage for the value
    dS = SU / jmax
    dt = T / imax

    S = numpy.zeros(jmax+1)
    t = numpy.zeros(imax+1)
    vNew = numpy.zeros(jmax+1)
    vOld = numpy.zeros(jmax+1)

    # calculate the values of S_j and t^i and check they work as expected
    for i in range(imax+1):
        t[i] = i*dt

    for j in range(jmax+1):
        S[j] = j*dS

    # first enter the value of the option at expiry
    for j in range(jmax+1):
        vOld[j] = max( X - S[j] , 0.0 )
        vNew[j] = max( X - S[j] , 0.0 )
        
    A_banded = numpy.zeros(shape=(3,jmax+1))
    u=1
    l=1
    l_and_u = (l, u)

    # b_0 V_0 + c_0 V_1 = d_0
    A_banded[1][0] = 1.0 # b_0
    A_banded[0][1] = 0.0 # c_0
    # populate middle rows
    for j in range(1,jmax):
        # a_j V_j-1 + b_j V_j + c_i V_j+1 = d_i
        A_banded[2][j-1] = 0.25*(sigma*sigma*j*j - r*j) # a_j
        A_banded[1][j] = -1./dt - 0.5*sigma*sigma*j*j - 0.5*r # b_j
        A_banded[0][j+1] = 0.25*(sigma*sigma*j*j + r*j) # c_j
    # populate last row
    # a_jmax V_jmax-1 + b_jmax V_jmax = d_jmax
    A_banded[2][jmax-1] = 0.0 # a_jmax 
    A_banded[1][jmax] = 1.0 # b_jmax
    
    # create a new copy of A_banded, that can be adjusted depending on the penalty
    A_banded_dash = numpy.copy(A_banded)

    for i in range(imax-1,-1,-1):
        # Create a vector for multiplication
        d = numpy.zeros(jmax+1)
        
        # adjust boundary condition to be P=X because it is American
        d[0] = X
        for j in range(1,jmax):
            aa = 0.25*(sigma*sigma*j*j - r*j) # a_j
            bb = 1./dt - 0.5*sigma*sigma*j*j - 0.5*r # b_j + 2./dt
            cc = 0.25*(sigma*sigma*j*j + r*j) # c_j
            d[j] = - aa*vOld[j-1] - bb*vOld[j] - cc*vOld[j+1]
        d[jmax] = 0.0
        
        # create a new copy of d, that can be adjusted depending on the penalty
        d_dash = numpy.copy(d)
            
        ## Penalty loop to solve for vNew
        for q in range(maxiter):
            
            # check which rows to adjust with a penalty
            for j in range(1,jmax):
                # check if P<X-S, if true apply penalty
                if vNew[j] < X - S[j]:
                    # b'_j = b_j - rho
                    A_banded_dash[1][j] = A_banded[1][j] - rho
                    # d'_j = d_j - rho*(X-S)
                    d_dash[j] = d[j] - rho*(X - S[j])
                    
                # otherwise revert to original A_banded and d
                else:
                    # b'_j = b_j
                    A_banded_dash[1][j] = A_banded[1][j] 
                    # d'_j = d_j 
                    d_dash[j] = d[j]
                    

            # Use banded solver solver to solve problem
            #    A' v^{i,q+1} = d'
            # here y ~ v_j^{i,q+1}
            y = solve_banded(l_and_u,A_banded_dash,d_dash)
            
            # get the change in value from the previous iteration
            # e = v_j^{i,q+1} - v_j^{i,q} 
            e = y-vNew
            
            # update the value of vNew
            vNew = y
            # check the norm against the tolerance for convergence
            if numpy.linalg.norm(e,1) < tol:
                print("Solved at step ",i,". Converged after ",q,"iterations.")
                # exit from the penalty loop
                break
        ##  solved for vNew
        
        vOld = numpy.copy(vNew)
        
    return S,vNew

In [ ]:
S,vNew = crankNicolson_AmerPut(10,1,0.05 , 0.4,50,50,50,1.e8,1.e-8,50)

# check a plot to see what it looks like
plt.xlim(0,20)
plt.plot(S,vNew)

# Tasks


1. Using interpolation to calculate the value at $S_0=X$ and with $S^U=50$, and different values of $imax$ and $jmax$, verify the convergence rate of this algorithm is $O((\Delta t)^2,(\Delta S)^2)$.


2. Try using Richardson extrapolation to improve your calculated results.


3. Using interpolation to calculate the value at $S_0=9.735$, is the convergence rate still $O((\Delta t)^2,(\Delta S)^2)$?


4. What is the efficiency of this method compared to the previous one? Say we want 6 digits of accuracy for the option value, which method can calculate it quicker? Can you use `numba` to compile and speed up your code where possible?


5. Update this code to include dividends, and make it solve for a call option.
